### Deep Models Under the GAN: Information Leakage from Collaborative Deep Learning

代码复现

## Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math
import time
import matplotlib.pyplot as plt
import random

%matplotlib inline

## Parameter settings

In [ ]:
#Parameters for collaborative learning
whether_warm_up = True
warm_up_epochs = 7
warm_up_batch_size = 256
warm_up_learning_rate = 0.001
warm_up_data_size = 600

Train_round = 200
Client_learning_rate = 0.0001
Client_batch_size = 256

Parameter_shape = []
Parameter_number = 0
Total_data_amount = 0

Test_acc = 0 #updated after each round
#Parameters for differential privicy
Whether_differential_privacy = False
Theta_d = 1
Theta_u = 0.9
Gamma = 0.003
Tua = 0.0001
Privacy_budget_per_para = 100


#Parameters for GAN
Whether_attack = True
Fake_label = 10
Attack_label = 3
Start_attack_acc = 0.9
Epoch_per_attack = 200
Noise_dimension = 100
Generated_picture_number = 256 #生成的图像数量，用于训练GAN
Generator_optimizer = tf.keras.optimizers.SGD(learning_rate=0.0003)
Discriminator_optimizer = tf.keras.optimizers.SGD(learning_rate=0.0003)

Num_examples_to_generate = 256 #生成的图像数量，用于引诱其他clients输出更多信息

Num_generate_for_show = 36 #生成的图像数量来展示

#Parameters for test

## Models

In [ ]:
# The discriminator model is the same as the collaboratively trained model
def make_discriminator_model():
  model = keras.Sequential()
  model.add(keras.layers.Conv2D(32, (5, 5), strides=(1, 1), padding='same', input_shape=[28, 28, 1]))
  model.add(keras.layers.Activation('tanh'))
  model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=None, padding='valid'))

  model.add(keras.layers.Conv2D(64, (5, 5), strides=(1, 1), padding='same'))
  model.add(keras.layers.Activation('tanh'))
  model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=None, padding='valid'))

  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(256,activation='tanh'))
  model.add(keras.layers.Dense(200,activation='tanh'))
  model.add(keras.layers.Dense(11))#activation=tf.nn.log_softmax))
  return model

# Malicious generator model
def make_generator_model():
  model = keras.Sequential()
    
  model.add(keras.layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.ReLU())

  model.add(keras.layers.Reshape((7, 7, 256)))
  assert model.output_shape == (None, 7, 7, 256)  # Batch size is not limited

  model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides=(1, 1), padding='same', use_bias=False))
  assert model.output_shape == (None, 7, 7, 128)
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.ReLU())

  model.add(keras.layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False))
  assert model.output_shape == (None, 14, 14, 64)
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.ReLU())

  model.add(keras.layers.Conv2DTranspose(1, (4, 4), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
  assert model.output_shape == (None, 28, 28, 1)

  return model

## Differential Privacy

In [ ]:
def flatten(W_or_G):
  flattened = np.array([])
  for i in range(len(W_or_G)):
    flattened = np.concatenate((flattened, W_or_G[i].flatten()))

  return flattened
def reshape(Flattened,para_shapes): 
#Flattened: a np.array of all parameters.
#para_shapes: a list of tulpes representing the number and shape of parameters of neural networks layers
  reshaped_Parameter = []
  for p_num,shape in para_shapes:
    reshaped_Parameter.append(np.reshape(Flattened[0:p_num],shape))
    Flattened = Flattened[p_num:]
  return reshaped_Parameter
  
def sigma(x,c,delta_f):
  return 2*c*delta_f/x
def differential_privacy(Gradient,privacy_budget_per_para,gamma,theta,tua,parameter_shape):
  #time1 = time.time()
  Gradient_flattened = flatten(Gradient)
  parameter_number = len(Gradient_flattened)
  c = floor(theta*parameter_number)
  epsilon = privacy_budget_per_para*c
  #print(epsilon,c,parameter_number,theta)
  epsilon_1 = 8/9*epsilon
  epsilon_2 = 2/9*epsilon
  sigma_1 = sigma(epsilon_1,c,2*gamma)
  sigma_2 = sigma(epsilon_2,c,2*gamma)
  Tua_with_noise = np.random.laplace(0,sigma_1,parameter_number)+tua
  R_w = np.random.laplace(0,2*sigma_1,parameter_number)
  Gradient_flattened[Gradient_flattened>gamma] = gamma
  Gradient_flattened[Gradient_flattened<-gamma] = -gamma
  Gradient_with_noise_flat1 = np.absolute(Gradient_flattened)+R_w
  index = []
  for i in range(parameter_number):
    if(Gradient_with_noise_flat1[i]>=Tua_with_noise[i]):
      index.append(i)
  if(len(index)>=c):
    np.random.shuffle(index)
    index = index[0:c]
  R_w2 = np.random.laplace(0,sigma_2,parameter_number)
  Gradient_with_noise_flat2 = R_w2+Gradient_flattened
  tmp = np.zeros(parameter_number)
  for i in index:
    tmp[i] = Gradient_with_noise_flat2[i]
  tmp = reshape(tmp,parameter_shape)
  #print("total time for DP:",time.time()-time1)
  return tmp

## Client

In [ ]:
cross_entropy = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def generator_loss(fake_output,attack_label):
  ideal_result = np.zeros(len(fake_output))+attack_label
  # for i in range(len(ideal_result)):
  #   # The class which attacker intends to get
  #   ideal_result[i] = attack_label
  return cross_entropy(ideal_result,fake_output)

#Discriminator needs to recognize the generated images
def discriminator_loss(fake_output):    
  fake_result = np.zeros(len(fake_output))+Fake_label
  return cross_entropy(fake_result, fake_output)

#Clients's behavior
class Client:
  def __init__(self,id):
    self.model = make_discriminator_model()
    self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=Client_learning_rate),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    server_Weight = server.send(1)
    self.model.set_weights(server_Weight)
    self.data = None
    self.labels = None
    self.data_amount = 0
    self.id = id

    self.malicious = False
    self.attack_label = None
    self.generator = None
    self.attack_round = 0
  def training_step(self):
    # global server
    Weight1 = self.model.get_weights()
    Weight2 = server.send(Theta_d)
    self.model.set_weights(Weight2)
    if(self.malicious==True and Test_acc>Start_attack_acc and self.model.evaluate(test_images,test_labels,verbose=0)[1] > Start_attack_acc):
      self.attack()
    
    self.model.fit(self.data,self.labels,validation_split=0,epochs=1,batch_size=Client_batch_size,verbose=1)
    Gradient = np.array(self.model.get_weights()) - np.array(Weight1)
    #start = time.time()
    if Whether_differential_privacy == True:
      return self.data_amount,differential_privacy(Gradient,Privacy_budget_per_para,Gamma,Theta_u,Tua,Parameter_shape)

    #print("Time spend on DP:",time.time()-start)
    return self.data_amount,Gradient
  def attack(self):
    print("Attack round ",self.attack_round,"begins.")
    self.attack_round = self.attack_round + 1
    Gen_loss = []
    Disc_loss = []
    for i in range(Epoch_per_attack):
      if i%100==0 :
        print(i)
      noise = tf.random.normal([Generated_picture_number,Noise_dimension])
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = self.generator(noise, training=True)
        fake_output = self.model(generated_images, training=False)
        gen_loss = generator_loss(fake_output,self.attack_label)
        Gen_loss.append(gen_loss)
        disc_loss = discriminator_loss(fake_output)
        Disc_loss.append(disc_loss)
      Gradient_of_generator = gen_tape.gradient(gen_loss,self.generator.trainable_variables)
      Generator_optimizer.apply_gradients(zip(Gradient_of_generator,self.generator.trainable_variables))
      Gradients_of_discriminator = disc_tape.gradient(disc_loss, self.model.trainable_variables)
      Discriminator_optimizer.apply_gradients(zip(Gradients_of_discriminator, self.model.trainable_variables))
    plt.plot(Gen_loss,label='Gen_loss')
    plt.plot(Disc_loss,label = 'Disc_loss')
    plt.legend()
    plt.show()
    noise = tf.random.normal([Num_examples_to_generate, Noise_dimension])
    
    generated_Image = self.generator(noise,training = False)
    malicious_Image = np.array(generated_Image)
    malicious_Label = np.array([self.labels[10]]*Num_examples_to_generate)
    self.data = self.data[0:self.data_amount]
    self.labels = self.labels[0:self.data_amount]
    self.data=np.concatenate((self.data,malicious_Image),axis = 0)
    self.labels=np.concatenate((self.labels,malicious_Label),axis = 0)
    self.generate_image(0)
  def generate_image(self,round):
    if self.malicious == False:
      assert 0,"I am innocent and would not generate any images"
    noise = tf.random.normal([Num_generate_for_show, Noise_dimension])
    generated_Image = self.generator(noise,training = False)
    fig = plt.figure(figsize=(6,6))
    for i in range(Num_generate_for_show):
      plt.subplot(6, 6, i+1)
      plt.imshow(generated_Image[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')
    plt.show()

## Server

In [ ]:
# Server's behavior
class Server:
  def __init__(self):
    self.model = make_discriminator_model()
    self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=warm_up_learning_rate),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])
    self.warm_up_data = None
    self.warm_up_labels = None
    self.warm_up_data_amount = 0
    self.stat = []
    for i in range(Parameter_number):
      self.stat.append([i,random.randint(0,5)])

  def warm_up(self):
    assert self.warm_up_data_amount != 0,"There is no data for warm up!"
    self.model.fit(self.warm_up_data,self.warm_up_labels,validation_split=0,epochs=warm_up_epochs,batch_size=warm_up_batch_size,verbose=1)

  #A participant uploads gradients
  def new_gradient(self,Gradient):
    Weight = self.model.get_weights()
    Weight2 = Weight + Gradient
    # Gradient_flattened = flatten(Gradient)
    # for i in range(len(Gradient_flattened)):
    #   if(Gradient_flattened[i] != 0):
    #     self.stat[i][1] = self.stat[i][1]+1
    self.model.set_weights(Weight2)
    
  #A participant downloads parameters.
  def send(self,theta):
    # largest_stat_index = [i[0] for i in sorted(self.stat,key = lambda x:(x[1],x[0]),reverse=True)[0:math.floor(theta*Parameter_number)]]
    # Weight=self.model.get_weights()
    # Weight_flattened = flatten(Weight)

    # selected_Weight = np.zeros(Parameter_number)
    # for i in largest_stat_index:
    #   selected_Weight[i] = Weight_flattened[i]
    # return selected_Weight,largest_stat_index
    #do not test the theta_d
    return self.model.get_weights()

## Collaboratively learning preparation

In [ ]:
#get Parameter number and shape
test_model = make_discriminator_model()
test_weight = test_model.get_weights()
for w in test_weight:
  shape = w.shape
  p_num = 1
  for i in shape:
    p_num = p_num*i
  Parameter_shape.append((p_num,w.shape))
  Parameter_number = Parameter_number + p_num 
del test_model,test_weight

#load data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5   # Normalization
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
test_images = (test_images - 127.5) / 127.5   # Normalization
Total_data_amount = len(train_labels)
###shuffle the data to get the warm up data
state = np.random.get_state()
np.random.shuffle(train_images)
np.random.set_state(state)
np.random.shuffle(train_labels)
warm_up_data = train_images[0:warm_up_data_size]
warm_up_labels = train_labels[0:warm_up_data_size]

#init server and clients
server = Server()
server.warm_up_data = warm_up_data
server.warm_up_labels = warm_up_labels
server.warm_up_data_amount = warm_up_data_size
if whether_warm_up == True:
  server.warm_up()
clients = []
for i in range(10):
  client = Client(i)
  client.data = train_images[train_labels==i]
  client.labels = train_labels[train_labels==i]
  client.data_amount = len(client.data)

  state = np.random.get_state()
  np.random.shuffle(client.data)
  np.random.set_state(state)
  np.random.shuffle(client.labels)
  clients.append(client)
del train_images,train_labels

if Whether_attack == True:
  clients[0].malicious = True
  clients[0].attack_label = Attack_label
  clients[0].generator = make_generator_model()

11493376/11490434 [==============================] - 0s 0us/step
Epoch 1/7
3/3 [==============================] - 31s 55ms/step - loss: 2.3008 - accuracy: 0.1700
Epoch 2/7
3/3 [==============================] - 0s 7ms/step - loss: 1.6713 - accuracy: 0.4917
Epoch 3/7
3/3 [==============================] - 0s 7ms/step - loss: 1.1349 - accuracy: 0.7333
Epoch 4/7
3/3 [==============================] - 0s 7ms/step - loss: 0.7949 - accuracy: 0.8033
Epoch 5/7
3/3 [==============================] - 0s 7ms/step - loss: 0.5789 - accuracy: 0.8433
Epoch 6/7
3/3 [==============================] - 0s 6ms/step - loss: 0.4509 - accuracy: 0.8817
Epoch 7/7
3/3 [==============================] - 0s 6ms/step - loss: 0.3635 - accuracy: 0.9017


## Training and attack

In [ ]:
total_start = time.time()
Test_loss = []
Test_accuracy = []
for i in range(Train_round):
  start = time.time()
  #Round robin
  for j in range(10):
    data_amount,Gradient = clients[j].training_step()
    Gradient = np.array(Gradient) * data_amount/Total_data_amount
    server.new_gradient(Gradient)

  end = time.time()
  print("Time for round:",i,"is ",end-start)
  test_loss, Test_acc = server.model.evaluate(test_images,test_labels,verbose=0)
  print("test_loss:",test_loss,"test_acc:",Test_acc)
  Test_loss.append(test_loss)
  Test_accuracy.append(Test_acc)
  plt.plot(Test_accuracy)
  plt.show()
print("total time:",time.time()-total_start) 